Установлю все, что нужно

In [ ]:
!pip install pymystem3==0.1.10
!pip install pymorphy2[fast]
!pip install razdel
!pip install gensim
!pip install nltk
!pip install rusenttokenize
!pip install regex

In [4]:
import string
from gensim.utils import tokenize
from gensim.summarization.textcleaner import split_sentences
from rusenttokenize import ru_sent_tokenize
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from nltk.corpus import stopwords
from string import punctuation
import re, os, json
mystem = Mystem()
morph = MorphAnalyzer()

Загружу файлик с текстом

In [5]:
from google.colab import files 
uploaded = files.upload()

Saving d1.txt to d1.txt


In [6]:
def remove_tags_3(text):
    no_tags_text = re.sub(r'<[^>]+>', ' ', text)
    no_space_sequences_text = re.sub('  +', ' ', no_tags_text)
    return no_space_sequences_text

In [7]:
text = open('d1.txt', encoding='UTF8')

In [11]:
text2 = text.readlines()

Избавлюсь от тегов

In [ ]:
for line in text2:
  print(remove_tags_3(line))

In [15]:
def hascyr(s):
  lower = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')
  return lower.intersection(s.lower()) != set()

In [16]:
lines = []
for line in text2:
  lines.append(remove_tags_3(line))

Удалю строки с кучей английских символов

In [17]:
text4 = []
for i in lines:
  if hascyr(i) == True:
    text4.append(i)

In [ ]:
for t in text4:
  print(t)

Соединю строки

In [ ]:
string = ''
for t in text4:
  string += t.split('\n')[0]
print(string)

Разобью на предложения

In [24]:
text5 = re.split('(?<=[\.?!]) +(?=[А-ЯЁ])', string.replace('\n', ' '))

In [ ]:
text6 = []
for m in text5:
  m = m.lower()
  text6.append(m)
print(text6)

In [26]:
import string
tt = str.maketrans(dict.fromkeys(string.punctuation))

In [ ]:
text7 = []
for k in text6:
  text7.append(k.translate(tt))
print(text7)

Удалю кавычки

In [ ]:
text8 = []
st = ''
for b in text7:
  for b2 in b:
    if b2 != '«' and b2 != '»':
      st += b2
  text8.append(st)
  st = ''
print(text8)

Посчитаю одинаковые предложения

In [29]:
from collections import Counter
sl = Counter(text8)

In [ ]:
print(Counter(text8))

In [ ]:
dict(Counter(text8).most_common(100))

Задание 2. Пункт 1.
Такие предложения есть, и их довольно много.
Вот примеры:

1.   свеча горела на столе свеча горела - 3
2.   потом пришла война разруха и долгодолго о тебе ни слуху не было ни духу - 2


In [35]:
string2 = ''
for slot in text8:
  string2 += slot.split('\n')[0]
list(razdel_tokenize(string2))[:10]

[Substring(1, 6, 'борис'),
 Substring(7, 17, 'леонидович'),
 Substring(18, 27, 'пастернак'),
 Substring(29, 35, 'доктор'),
 Substring(36, 42, 'живаго'),
 Substring(44, 50, 'доктор'),
 Substring(51, 57, 'живаго'),
 Substring(59, 67, 'итоговое'),
 Substring(68, 80, 'произведение'),
 Substring(81, 87, 'бориса')]

In [41]:
list3 = []
for slot2 in [token.text for token in list(razdel_tokenize(string2))]:
  if len(slot2) > 6:
    list3.append(slot2)
dict(Counter(list3).most_common(10))


{'андреевич': 273,
 'андреевича': 65,
 'говорит': 65,
 'доктора': 71,
 'которой': 76,
 'которые': 86,
 'который': 103,
 'которых': 66,
 'несколько': 103,
 'человек': 87}

Задание 2. Пункт 2. Наиболее частый токен в тексте: андреевич - 273 повторения

In [42]:
from nltk.stem.snowball import SnowballStemmer

In [43]:
stemmer = SnowballStemmer('russian')

In [ ]:
for word in [token.text for token in list(razdel_tokenize(string2))]:
  if len(word) > 4 and stemmer.stem(word) == word:
    print(word, ' ', stemmer.stem(word))

Задание 3. Пункт 2. Примеры:


1.   Отпор
2.   тронув



Для полного списка запустите код 46





In [53]:
values = {}
count = 0
for word in [token.text for token in list(razdel_tokenize(string2))]:
  if count < 10:
    if stemmer.stem(word) in values:
      if str(values[stemmer.stem(word)]) != word:
        print('word1: ', str(values[stemmer.stem(word)]), ' word2: ', word, ' str: ', stemmer.stem(word))
        count += 1
    else:
      values.setdefault(stemmer.stem(word), word)


word1:  борис  word2:  бориса  str:  борис
word1:  пастернак  word2:  пастернака  str:  пастернак
word1:  которой  word2:  которого  str:  котор
word1:  всей  word2:  все  str:  все
word1:  доктор  word2:  доктора  str:  доктор
word1:  которой  word2:  который  str:  котор
word1:  пастернак  word2:  пастернака  str:  пастернак
word1:  мире  word2:  миром  str:  мир
word1:  доктор  word2:  доктора  str:  доктор
word1:  стоило  word2:  стоит  str:  сто


Задание 2. Пункт 1. Примеры:



1.   всей, все привелось к "все"
2.   стоило, стоит привелось к "сто"



In [58]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [59]:
from nltk.corpus import stopwords

In [ ]:
print(stopwords.words('russian'))

Проверю, каких слов нет в списке

In [73]:
if 'около' not in stopwords.words('russian'):
  print('no')
else:
  print('yep')

no


который/которое/которые - не несет в себе смысловой нагрузки

однако - часто используется, как ввожное

наверное - тоже вводное

ок - заимствованные слова, тоже не несут нагрузки

около - предлоги, тоже можно вынести в стоп-слова



In [75]:
print(string2)

 борис леонидович пастернак  доктор живаго  доктор живаго  итоговое произведение бориса пастернака книга всей его жизниэтот роман принес его автору мировую известность и нобелевскую премию присуждение которой обернулось для поэта оголтелой политической травлей обвинениями в измене родине и в результате стоило ему жизни  доктор живаго  роман сама ткань которого убедительнее свидетельствует о чуде чем все размышления доктора и обобщения авторачеловек который так пишет бесконечно много пережил и передумал и главные его чувства на свете  восхищенное умиление и слезное сострадание конечно есть в его мире место и презрению и холодному отстранению  но не в них сутьроман пастернака  оплакивание прежних заблуждений и их жертв те кто не разделяет молитвенного восторга перед миром достойны прежде всего жалостиперечитывать доктора живаго стоит именно тогда когда кажется что жить не стоиттогда десять строк из этого романа могут сделать то же что делает любовь в одном из стихотворений доктора жизнь 

In [87]:
mystem.lemmatize(string2)[:40]

[' ',
 'борис',
 ' ',
 'леонидович',
 ' ',
 'пастернак',
 '  ',
 'доктор',
 ' ',
 'живаго',
 '  ',
 'доктор',
 ' ',
 'живаго',
 '  ',
 'итоговый',
 ' ',
 'произведение',
 ' ',
 'борис',
 ' ',
 'пастернак',
 ' ',
 'книга',
 ' ',
 'весь',
 ' ',
 'его',
 ' ',
 'жизниэтоты',
 ' ',
 'роман',
 ' ',
 'приносить',
 ' ',
 'он',
 ' ',
 'автор',
 ' ',
 'мировой']

In [ ]:
pip install pymorphy2[fast]

In [79]:
words_analized = [morph.parse(token) for token in [token.text for token in list(razdel_tokenize(string2))]]

In [86]:
count = 0
while count < 20:
  print(words_analized[count][0].normal_form)
  print('\n')
  count += 1

борис


леонидович


пастернак


доктор


живаго


доктор


живаго


итоговый


произведение


борис


пастернак


книга


весь


он


жизниэтота


роман


принести


он


автор


мировой




В лекции в написали, что mystem лучше, чем pymorphy, но на этих данных pymorphy показал результат лучше, чем mystem



1.   Pymorphy сохранил прошедшее время глагола "принес"
2.   Pymorphy одинакого интерпретировал "его" в тексте

